### Lets build chatbots using langchain

In [1]:
from langchain_groq import ChatGroq
from langchain.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
model = ChatGroq(model="llama-3.1-8b-instant")

In [3]:
# lets invoke the model
model.invoke([HumanMessage(content="Hi, my name is srini!")])

AIMessage(content="Nice to meet you, Srini! I'm happy to chat with you. Is there something I can help you with, or would you like to just say hello and see where the conversation goes?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 43, 'total_tokens': 84, 'completion_time': 0.060703172, 'completion_tokens_details': None, 'prompt_time': 0.002079811, 'prompt_tokens_details': None, 'queue_time': 0.005433615, 'total_time': 0.062782983}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9ca2574dca', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--1b4b56bf-fe9a-4ab0-98b7-803ce0e1164c-0', usage_metadata={'input_tokens': 43, 'output_tokens': 41, 'total_tokens': 84})

In [4]:
model.invoke([HumanMessage(content="What is my name?")])

AIMessage(content="I don't have any information about your name. I'm a large language model, I don't have the ability to retain personal information or recall previous conversations. Each time you interact with me, it's a new conversation and I don't have any prior knowledge about you. If you'd like to share your name with me, I'd be happy to chat with you!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 40, 'total_tokens': 116, 'completion_time': 0.093150409, 'completion_tokens_details': None, 'prompt_time': 0.001846363, 'prompt_tokens_details': None, 'queue_time': 0.005630164, 'total_time': 0.094996772}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9ca2574dca', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--8f40840e-046e-4f70-b889-2dae0c7f682c-0', usage_metadata={'input_tokens': 40, 'output_tokens': 76, 'total_tokens': 116})

In [5]:
# lets invoke the model
model.invoke(
    [
        HumanMessage(content="Hi, my name is srini!"),
        AIMessage(content="Nice to meet you, Srini! I'm happy to chat with you. Is there something on your mind that you'd like to talk about, or do you just want to say hello?"),
        HumanMessage(content="What is my name?")
    ]
)

AIMessage(content='Your name is Srini.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 97, 'total_tokens': 104, 'completion_time': 0.009091895, 'completion_tokens_details': None, 'prompt_time': 0.005581137, 'prompt_tokens_details': None, 'queue_time': 0.005117414, 'total_time': 0.014673032}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9ca2574dca', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--b358abe8-b46e-4914-95db-5be4c2e627e1-0', usage_metadata={'input_tokens': 97, 'output_tokens': 7, 'total_tokens': 104})

As we see here:
- If we do not provide the context. LLM will not have any information
- If we provide the context, it can easily know and answer based on context

This is called illusion of memory

### Message History

We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [6]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# history store
store = {}

In [7]:
# create a function to get the chat history based on session id

def get_chat_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [8]:
# setup config
config = {'configurable': {'session_id': 'chat_1'}}

# setup model with the session history
with_chat_history = RunnableWithMessageHistory(model, get_session_history=get_chat_history)

In [9]:
# interact with model with session history
response = with_chat_history.invoke(
    [
        HumanMessage(content="Hi, my name is srini! I am Senior GenAI Engineer!"),
    ],
    config=config
)

In [10]:
response.content

"Hello Srini, Senior GenAI Engineer! It's great to meet you. I'm a large language model, and I'm here to assist and learn from you. What brings you here today? Are you working on a new AI project, or do you need help with something specific?"

In [11]:
# change the config - new session id
config1 = {'configurable': {'session_id': 'chat_2'}}

# interact with model with session history
response = with_chat_history.invoke(
    [
        HumanMessage(content="Hi, What is my name?"),
    ],
    config=config1
)

response.content

"I'm happy to chat with you, but I don't have any information about your name. I'm a large language model, I don't have the ability to retain information about individual users or their personal details. Each time you interact with me, it's a new conversation and I don't retain any context or information from previous conversations.\n\nIf you'd like to share your name with me, I'd be happy to chat with you and use it in our conversation. Or, if you'd prefer, we can keep our conversation anonymous and use pronouns or a username instead. Let me know what you prefer!"

In [12]:
# interact with model with session history - with the first session
response = with_chat_history.invoke(
    [
        HumanMessage(content="Hi, What is my name?"),
    ],
    config=config
)

response.content

"Your name is Srini, and you're a Senior GenAI Engineer."

### Prompt templates

Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the question to the best of your ability."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [14]:
# we have messages placeholder setup to accept "messages"
# so, when we invoke the chain we need to pass the messages to the chain

chain.invoke({
    "messages": [
        HumanMessage(content="Hi, My name is srini!"),
    ]
})

AIMessage(content="Hello Srini, I'm glad you're here. How can I assist you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 60, 'total_tokens': 79, 'completion_time': 0.035467118, 'completion_tokens_details': None, 'prompt_time': 0.003287779, 'prompt_tokens_details': None, 'queue_time': 0.005474608, 'total_time': 0.038754897}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9ca2574dca', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--e5428493-ee25-47bc-9cd6-1b32efdb5b32-0', usage_metadata={'input_tokens': 60, 'output_tokens': 19, 'total_tokens': 79})

In [15]:
# let us set the chain with session history, by providing chain with the session history
# chain = prompt | model

chain_with_history = RunnableWithMessageHistory(chain, get_session_history=get_chat_history)

In [16]:
# setup new config 
config2 = {'configurable': {'session_id': 'chat_3'}}

# interact with model with session history
response = chain_with_history.invoke(
    [HumanMessage(content="Hi, My name is srini!")],
    config=config2
)

response.content

"Hello Srini! It's nice to meet you. I'm here to help with any questions or topics you'd like to discuss. How's your day going so far?"

When you call chain_with_history.invoke(), the RunnableWithMessageHistory wrapper:

1. Takes your input messages: [HumanMessage(content="Hi, My name is srini!")]
2. Retrieves the chat history from the session (via get_chat_history)
3. Combines them together
4. Passes the combined messages to the underlying chain in the format it expects: {"messages": [history... + new messages...]}
5. Saves the new messages to the history after getting the response

So you are effectively sending messages - the RunnableWithMessageHistory wrapper is just doing the work of formatting them and managing the history for you
automatically!

In [27]:
# adding more complexity to the chain
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the question to the best of your ability in {language} Language."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [28]:
# first lets directly call chain
chain.invoke({
    "messages": [
        HumanMessage(content="Hi, My name is srini!"),
    ],
    "language": "Hindi"
})

AIMessage(content='नमस्ते Srini! मुझे खुशी है आपकी मुलाकात करने का मौका मिला है। मैं आपकी सहायता के लिए यहाँ हूँ। क्या आपके पास कोई प्रश्न या समस्या है जिससे मैं आपकी मदद कर सकता हूँ?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 63, 'total_tokens': 147, 'completion_time': 0.112864252, 'completion_tokens_details': None, 'prompt_time': 0.003542867, 'prompt_tokens_details': None, 'queue_time': 0.005432938, 'total_time': 0.116407119}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9ca2574dca', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--33461484-0718-4657-ae91-9f487b9e829a-0', usage_metadata={'input_tokens': 63, 'output_tokens': 84, 'total_tokens': 147})

In [ ]:
chain_with_history_v2 = RunnableWithMessageHistory(chain, get_session_history=get_chat_history, input_messages_key="messages")

In [ ]:
config3 = {'configurable': {'session_id': 'chat_4'}}

response = chain_with_history_v2.invoke(
    {
        "messages": [
            HumanMessage(content="Hi, My name is srini!"),
        ],
        "language": "Hindi",
    },
    config=config3
)

response.content

'नमस्ते!  मैं आपका सहायक हूँ, और मैं आपकी सहायता के लिए तैयार हूँ। आपका नाम स्रीनि है (Srini), मैं आपके साथ बात करने के लिए उत्साहित हूँ। क्या आपके पास कोई प्रश्न है या क्या आप कुछ जानना चाहते हैं?'

In [ ]:
response = chain_with_history_v2.invoke(
    {
        "messages": [
            HumanMessage(content="What is my name?"),
        ],
        "language": "Hindi",
    },
    config=config3
)

response.content

'आपका नाम स्रीनि (Srini) है ।'

In chain_with_history - which is a wrapper of RunnableWithMessageHistory (combining chain and session history) 
- assumes that the whole input is "messages" that is specified in ChatPromptTemplate's MessagesPlaceholder

In chain_with_history - which is a wrapper of RunnableWithMessageHistory (combining chain and session history) 
- it also has `input_messages_key="messages"`
- this DOES NOT assumes that the whole input is "messages" that is specified in ChatPromptTemplate's MessagesPlaceholder
- need to specify "messages" and pass all the required messages here such has HumanMessage etc
- advantage is that we can pass other variables present in chat prompt template such as "language"